# LlamaParse with GPT-4o


<a href="https://colab.research.google.com/github/run-llama/llama_parse/blob/main/examples/test_tesla_impact_report/test_gpt4o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPT-4o is a [fully multimodal model by OpenAI](https://openai.com/index/hello-gpt-4o/) released in May 2024. It matches GPT-4 Turbo performance in text and code, and has significantly improved vision and audio capabilities.

The expanded vision/audio capabilities mean that it can be used for document parsing, by treating each page as an image and performing document extraction. We support using GPT-4o natively in LlamaParse for document parsing. The notebook below walks you through an example of using GPT-4o over the Tesla impact report.

**NOTE**: The pricing for LlamaParse + gpt4o is an order more expensive than using LlamaParse by default. Currently, every page parsed with gpt4o counts for 10 pages in the LlamaParse usage tracker.


In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os

In [ ]:
os.environ["LLAMA_CLOUD_API_KEY"] = "<LLAMA_CLOUD_API_KEY>

### Use LlamaParse with `gpt4o_mode=True`

By turning on gpt4o, we use GPT-4o multimodal capabilities to do document parsing per page instead of the LlamaParse default pipeline.

We load a snippet of the [2019 Tesla impact report](https://www.tesla.com/ns_videos/2019-tesla-impact-report.pdf). **NOTE**: The report is 57 pages, but will count for 570 pages in LlamaParse due to GPT-4o usage (which is approximately $1.71 USD).

You can optionally choose to provide a `gpt4o_api_key`. If you do this, then we will use your API key to make GPT-4o calls, and your LlamaParse usage will be counted as if `gpt4o_mode` was not turned on (each page will be counted as a page instead of 10 pages). 

In [ ]:
!wget "https://www.dropbox.com/scl/fi/vu6w1dsfo5eddydz13ssm/2019-tesla-impact-report-15.pdf?rlkey=ik8lfqbg2p1ervss4qqt3xose&st=70j04z8j&dl=1" -O "2019-tesla-impact-report-15.pdf"

--2024-05-21 00:10:32--  https://www.dropbox.com/scl/fi/vu6w1dsfo5eddydz13ssm/2019-tesla-impact-report-15.pdf?rlkey=ik8lfqbg2p1ervss4qqt3xose&st=70j04z8j&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6057:18::a27d:d12, 162.125.13.18
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6057:18::a27d:d12|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc872df1ff4ea2fecd3d024fa97a.dl.dropboxusercontent.com/cd/0/inline/CTTnZs8U4V1GtUCNxoB7INwmLq2yU97Q6QbWS6uVnb_XdHe368GrqF0zLDEKTnpc-x7utwNUUpMvWjLyrujrqNVrbGKTKa6hwHu5BxYPA2zXYrzdAEZyeve274xpHZKFywQ/file?dl=1# [following]
--2024-05-21 00:10:33--  https://uc872df1ff4ea2fecd3d024fa97a.dl.dropboxusercontent.com/cd/0/inline/CTTnZs8U4V1GtUCNxoB7INwmLq2yU97Q6QbWS6uVnb_XdHe368GrqF0zLDEKTnpc-x7utwNUUpMvWjLyrujrqNVrbGKTKa6hwHu5BxYPA2zXYrzdAEZyeve274xpHZKFywQ/file?dl=1
Resolving uc872df1ff4ea2fecd3d024fa97a.dl.dropboxusercontent.com (uc872df1ff4ea2fecd3d024fa97a.dl.dropboxusercontent.com)... 2

In [ ]:
from llama_parse import LlamaParse

parser_gpt4o = LlamaParse(
    result_type="markdown",
    # api_key=api_key,
    gpt4o_mode=True,
    split_by_page=True,
    # gpt4o_api_key="<gpt4o_api_key>"
)

In [ ]:
documents_gpt4o = parser_gpt4o.load_data("./2019-tesla-impact-report-15.pdf")

Started parsing the file under job_id bf7d4619-3e26-479d-80e9-25702186ef32
.

In [ ]:
print(documents_gpt4o[3].get_content())

# Mission & Tesla Ecosystem

Climate change is reaching alarming levels in large part due to emissions from burning fossil fuels for transportation and electricity generation. In 2016, carbon dioxide (CO2) concentration levels in the atmosphere exceeded the 400 parts per million threshold on a sustained basis - a level that climate scientists believe will have a catastrophic impact on the environment. Worse, annual global CO2 emissions continue to increase and have approximately doubled over the past 50 years to over 43 gigatons in 2019. The world’s current path is unwise and unsustainable.

The world cannot reduce CO2 emissions without addressing both energy generation and consumption. And the world cannot address its energy habits without first directly reducing emissions in the transportation and energy sectors. We are focused on creating a complete energy and transportation ecosystem from solar generation and energy storage to all-electric vehicles that produce zero tailpipe emissi

## Build RAG pipeline over the Parsed Report

We now try building a RAG pipeline over this parsed report. It's not a lot of text, but we split it into chunks and load it into a simple in-memory vector store.

We ask a question over the parsed markdown table and get back the right answer! We also ask a question over the text.

In [ ]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(documents_gpt4o)

In [ ]:
query_engine = vector_index.as_query_engine(similarity_top_k=6)

In [ ]:
response = query_engine.query(
    "What are the greenhouse emissions for agriculture and transportation?"
)

In [ ]:
print(str(response))

Agriculture accounts for 20% of global greenhouse gas emissions, while transportation contributes 16% of these emissions.


Let's also try asking a question over another piece of the text.

In [ ]:
response = query_engine.query(
    "How does the EPA range of Teslas compare with other vehicles? Give details"
)

In [ ]:
print(str(response))

The EPA range of Tesla vehicles varies across different models. The Model 3 Standard Range Plus (SR+) achieves an EPA range of 4.8 miles/kWh, making it the most efficient electric vehicle in production. The Model Y all-wheel drive (AWD) achieves 4.1 miles/kWh, which positions it as the most efficient electric SUV produced to date. The energy efficiency of Tesla vehicles is highlighted by these EPA range figures, showcasing their advancements in powertrain efficiency compared to other electric vehicles on the market.


In [ ]:
print(response.source_nodes[0].get_content())

# Reducing Carbon Footprint Even Further
## Improving Powertrain Efficiency

Tesla vehicles are known to have the highest energy efficiency of any EV built to date. In the early days of Model S production, we were able to achieve energy efficiency of 3.1 EPA miles / kWh. Today, our most efficient Model 3 Standard Range Plus (SR+) achieves an EPA range of 4.8 miles / kWh, more than any EV in production. Model Y all-wheel drive (AWD) achieves 4.1 EPA miles / kWh, which makes it the most efficient electric SUV produced to date.

The energy efficiency of Tesla vehicles will continue to improve further over time as we continue to improve our technology and powertrain efficiency. It is also reasonable to assume that our high-mileage products, such as our future Tesla Robotaxis, will be designed for maximum energy efficiency as handling, acceleration, and top speed become less relevant. That way, we will minimize cost for our customers as well as reduce the carbon footprint per mile driven.

